In [1]:
from glob import glob
import uproot as ur
import numpy as np
import json, os
import os.path as osp
from tqdm.auto import tqdm
from json import load
from phc import module_reload, plot_hist
module_reload('zhh')
from zhh import parse_sample_path, get_preselection_passes, ProcessIndex
version = 'v1'

REPO_ROOT = '/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH'
DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/PreselectionFinal/{version}'
INDEX_DIR = '/nfs/dust/ilc/user/bliewert/zhh/CreateRawIndex/v1'

processes = np.load(f'{INDEX_DIR}/processes.npy')
samples = np.load(f'{INDEX_DIR}/samples.npy')
chunks = np.load(f'{DATA_ROOT}/../../CreatePreselectionChunks/v1/chunks.npy')

In [8]:
a = glob('/nfs/dust/ilc/user/bliewert/zhh/PreselectionFinal/v1/stdall_*.txt')
a.sort()

Validate chunks

In [5]:
for chunk in chunks:
    with open(f'{DATA_ROOT}/{chunk["branch"]}_FinalStateMeta.json') as metaf:
        meta = load(metaf)
        
        if meta['nEvtSum'] != chunk['chunk_size']:
            raise Exception(f"Chunk mismatch for branch <{chunk['branch']}> : {meta['nEvtSum']} vs {chunk['chunk_size']}")

Exception: Chunk mismatch for branch <0> : 3583 vs 3600

In [5]:
files = ['_PreSelection_llHH.root',
         '_PreSelection_vvHH.root',
         '_PreSelection_qqHH.root',
         '_FinalStates.root',
         '_FinalStateMeta.json',
         '.slcio',
         '_Source.txt']

sel_chunks = chunks[chunks['chunk_start'] == 0]
to_delete = []
for chunk in sel_chunks:
    for f in files:
        path = f'{DATA_ROOT}/{chunk["branch"]}{f}'
        if osp.isfile(path):
            to_delete.append(path)

for p in tqdm(to_delete):
    os.remove(p)

In [5]:
chunks[chunks['branch'] == 12902]

array([(12902, 'eeveev', 'eeveev_RR', '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/6f_eeWW/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I108622.Peeveev.eR.pR.n001.d_dstm_10322_0.slcio', 0, 0, 1)],
      dtype=[('branch', '<i4'), ('process', '<U60'), ('proc_pol', '<U64'), ('location', '<U512'), ('n_chunks', '<i4'), ('chunk_start', '<i4'), ('chunk_size', '<i4')])

Introduce indices on samples and processes (included in future runs)

In [12]:
dtype_sample = ProcessIndex.dtype_sample
dtype_process = ProcessIndex.dtype_process

if not 'sid' in samples.dtype.names:
    samples_new = np.empty(len(samples), dtype=dtype_sample)
    samples_new['sid'] = np.arange(len(samples))
    for col in samples.dtype.names:
        samples_new[col] = samples[col]
        
    if len(samples) == len(samples_new):
        np.save(f'{INDEX_DIR}/samples.npy', samples_new)

if not 'pid' in processes.dtype.names:
    #processes_new = np.array([pids, *(processes[col] for col in processes.dtype.names)], dtype=dtype_process)
    processes_new = np.empty(len(processes), dtype=dtype_process)
    processes_new['pid'] = np.arange(len(processes))
    for col in processes.dtype.names:
        processes_new[col] = processes[col]

    if len(processes) == len(processes_new):
        np.save(f'{INDEX_DIR}/processes.npy', processes_new)
        
print('Conversion successful')

Conversion successful


Prototyping

In [16]:
results = get_preselection_passes(DATA_ROOT)

  0%|          | 0/15497 [00:00<?, ?it/s]

In [17]:
np.save(f'{REPO_ROOT}/preselection.npy', results)

In [10]:
results = np.load(f'{REPO_ROOT}/preselection.npy')

In [11]:
results

array([('2f_z_bhabhag', -1, -1,   92841, 2.96197998e+03, 1.78999996e+01,  387,   0,    0, 2.00993900e+01, '2f_z_bhabhag_LL'),
       ('2f_z_bhabhag', -1,  1,  169230, 3.64515991e+03, 2.27000008e+01,  659,   0,    0, 2.52014256e+01, '2f_z_bhabhag_LR'),
       ('2f_z_bhabhag',  1, -1,   11745, 3.37210010e+03, 2.02999992e+01,   57,   0,    0, 2.00976601e+01, '2f_z_bhabhag_RL'),
       ('2f_z_bhabhag',  1,  1,   19125, 2.95691992e+03, 1.78999996e+01,   85,   0,    0, 2.00993252e+01, '2f_z_bhabhag_RR'),
       ('2f_z_bhabhang', -1, -1, 2966452, 1.23911102e+05, 7.24000015e+01, 3252,   0,    0, 2.63156109e+01, '2f_z_bhabhang_LL'),
       ('2f_z_bhabhang', -1,  1, 2984564, 1.33070797e+05, 7.84000015e+01, 3772,   0,    0, 5.21660233e+01, '2f_z_bhabhang_LR'),
       ('2f_z_bhabhang',  1, -1,  453567, 1.30234703e+05, 7.51999969e+01,  533,   0,    0, 2.00994110e+01, '2f_z_bhabhang_RL'),
       ('2f_z_bhabhang',  1,  1,  801472, 1.23916500e+05, 7.00999985e+01,  840,   0,    0, 2.00994492e+01, '2f_z

In [12]:
np.unique(results['process'])

array(['2f_z_bhabhag', '2f_z_bhabhang', '2f_z_h', '2f_z_l', '2f_z_nung',
       '4f_lowmee_sze_l', '4f_lowmee_szeorsw_l', '4f_sw_l', '4f_sw_sl',
       '4f_sze_l', '4f_sze_sl', '4f_szeorsw_l', '4f_sznu_l', '4f_sznu_sl',
       '4f_ww_h', '4f_ww_l', '4f_ww_sl', '4f_zz_h', '4f_zz_l', '4f_zz_sl',
       '4f_zzorww_h', '4f_zzorww_l', 'e1e1hh', 'e1e1qqh', 'e2e2hh',
       'e2e2qqh', 'e3e3hh', 'e3e3qqh', 'eeeeee', 'eeeell', 'eeeexx',
       'eeeeyy', 'eellxx', 'eellyy', 'eeveev', 'eevelv', 'eeveyx',
       'eevlev', 'eevllv', 'eevlyx', 'eexyev', 'eexylv', 'eexyyx',
       'llllee', 'llllll', 'llvelv', 'llveyx', 'llvlev', 'llvllv',
       'llvlyx', 'llxyev', 'llxylv', 'llxyyx', 'n1n1hh', 'n1n1qqh',
       'n23n23hh', 'n23n23qqh', 'qqhh', 'qqqqh', 'vvveev', 'vvvelv',
       'vvveyx', 'vvvlev', 'vvvllv', 'vvvlyx', 'vvvvxx', 'vvvvyy',
       'vvxyev', 'vvxylv', 'vvxyyx', 'xxveev', 'xxvelv', 'xxveyx',
       'xxvlev', 'xxvllv', 'xxvlyx', 'xxxxee', 'xxxxll', 'xxxxvv',
       'xxxxxx', 'xxxyev', 'x

In [15]:
from ast import literal_eval as make_tuple
import json

def test_meta_files(DATA_ROOT:str=DATA_ROOT)->bool:
    files = glob(f'{DATA_ROOT}/*_Source.txt')
    #branches = list(map(lambda x: x.split(f'{DATA_ROOT}/')[1].split('_Source.txt')[0], files))

    for f in tqdm(files):
        branch = f.split(f'{DATA_ROOT}/')[1].split('_Source.txt')[0]
        
        if osp.isfile(f'{DATA_ROOT}/{branch}_FinalStateMeta.json'):
            with open(f, 'r') as file:
                src_spec = file.read()
                if src_spec.startswith('('):
                    src_file, chunk_start, chunk_end = make_tuple(src_spec)
                else:
                    src_file = src_spec
            
            # Read metadata
            with open(f'{DATA_ROOT}/{branch}_FinalStateMeta.json', 'r') as file:
                meta = json.load(file)

            n_gen = meta['nEvtSum']
            proc = meta["processName"]
            
            if proc == '' or n_gen == 0:
                print(src_file)
                raise Exception(branch)
    
    return True

In [16]:
test_meta_files()

  0%|          | 0/14449 [00:00<?, ?it/s]

True

In [ ]:
for entry in results:
    print(f'{entry["proc_pol"]} | {entry["n_gen"]} events | wt: {entry["weight"]} | {entry["n_pass_llhh"]} : {entry["n_pass_vvhh"]} : {entry["n_pass_qqhh"]}')
    
#np.save(f'{REPO_ROOT}/results.npy', results)

Preselection Detailed

In [29]:
a = ur.open(f'{DATA_ROOT}/{branch}_PreSelection_{presel}.root:eventTree')

In [13]:
from typing import Union
import awkward as ak

if True:
    DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/PreselectionFinal/{version}'
    branch = 1
    presel = 'llHH'
else:
    DATA_ROOT = f'/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH/output'
    branch = 'zhh'
    presel = 'llHH'

def analyze_presel_file(DATA_ROOT:str, branch:int, presel):
    branch = str(branch)
    
    with ur.open(f'{DATA_ROOT}/{branch}_PreSelection_{presel}.root') as rf:
        if presel == 'llHH':
            pass_nIsoLeptons = np.array(rf['eventTree']['nIsoLeptons'].array()) == 2

            lepTypes = rf['eventTree']['lepTypes'].array()
            
            # (excactly) two electron/muon-like signatures:
            pass_ltype11 = np.sum(np.abs(lepTypes) == 11, axis=1) == 2
            pass_ltype13 = np.sum(np.abs(lepTypes) == 13, axis=1) == 2
            
            # eq: print(np.sum(lepTypes, axis=1))
            
            dileptonMassDiff = np.array(rf['eventTree']['dileptonMassDiff'].array())
            pass_dileptonMassDiff = dileptonMassDiff < 40.
            
            
        elif presel == 'qqHH':
            pass
        elif presel == 'vvHH':
            pass
        
    #plot_hist(dileptonMass, xlim=[0,500]);

analyze_presel_file(DATA_ROOT, branch, presel)

In [2]:
if True:
    DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/PreselectionFinal/{version}'
    branch = 1
    presel = 'llHH'
else:
    DATA_ROOT = f'/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH/output'
    branch = 'zhh'
    presel = 'llHH'

In [3]:
a = ur.open(f'{DATA_ROOT}/{branch}_FinalStates.root')
b = a['eventTree']

In [5]:
b.keys()

['run',
 'event',
 'error_code',
 'final_states',
 'final_state_counts',
 'final_state_counts/final_state_counts.first',
 'final_state_counts/final_state_counts.second',
 'process',
 'event_category',
 'event_category_zhh',
 'n_fermion',
 'n_higgs']

In [13]:
b['process'].array()

<Array [3171, 3171, 3171, 3171, ..., 3171, 3171, 3171] type='3582 * int32'>

In [30]:
b = ur.open(f'{DATA_ROOT}/{branch}_PreSelection_llHH.root:eventTree')

In [41]:
glob(f'{DATA_ROOT}/*_FinalStateMeta.json')[10]

'/nfs/dust/ilc/user/bliewert/zhh/PreselectionFinal/v1/10027_FinalStateMeta.json'

In [46]:
chunks.dtype.names

('branch',
 'process',
 'proc_pol',
 'location',
 'n_chunks',
 'chunk_start',
 'chunk_size')

In [48]:
def get_chunks_factual(DATA_ROOT:str, chunks_in:np.ndarray):
    dtype_new = np.dtype(chunks_in.dtype.descr + [('chunk_size_factual', 'i')])
    chunks = np.zeros(chunks_in.shape, dtype_new)
    
    for name in chunks_in.dtype.names:
        chunks[name] = chunks_in[name]
    
    branches = chunks['branch']
    for branch in branches:
        with open(f'{DATA_ROOT}/{branch}_FinalStateMeta.json') as jf:
            meta = json.load(jf)
            n_events = meta['nEvtSum']
            
        chunks['chunk_size_factual'][chunks['branch'] == branch] = n_events
        
    return chunks
            

In [49]:
chunks_factual = get_chunks_factual(DATA_ROOT, chunks)

In [58]:
from typing import Union
import awkward as ak

if True:
    DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/PreselectionFinal/{version}'
    branch = 1
    presel = 'llHH'
else:
    DATA_ROOT = f'/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH/output'
    branch = 'zhh'
    presel = 'llHH'

def presel_stack(DATA_ROOT:str,
                 samples,
                 processes,
                 chunks_f,
                 branches:list,
                 kinematics:bool=False):   
            
    dtype = [
        ('sid', 'i'),
        ('pid', 'i'),
        ('event', 'i'),
        ('event_category', 'i'),
        ('xx_thrust', 'f'),
        ('xx_e_vis', 'f'),
        ('xx_pt_miss', 'f'),
        ('xx_nisoleps', 'i'),
        
        ('ll_pass', 'i'),
        ('vv_pass', 'i'),
        ('qq_pass', 'i'),
    ]
    
    if kinematics:
        for dt in [
            # llHH
            ('ll_dilepton_type', 'i'),
            ('ll_dilepton_mass_diff', 'f'),
            ('ll_nbjets', 'i'),
            
            ('ll_dijet_h1_mass_diff', 'f'),
            ('ll_dijet_h2_mass_diff', 'f'),
            
            # vvHH
            ('vv_mhh', 'f'),
            ('vv_mh1', 'f'),
            ('vv_mh2', 'f'),
            ('vv_nbjets', 'i'),
            
            ('vv_dijet_h1_mass_diff', 'f'),
            ('vv_dijet_h2_mass_diff', 'f'),
            
            # qqHH
            ('qq_mh1', 'f'),
            ('qq_mh2', 'f'),
            ('qq_nbjets', 'i')
        ]:
            dtype.append(dt)
    
    r_size = chunks_f[chunks_f['branch'] == branches]['chunk_size_factual'].sum()
    results = np.empty(r_size, dtype=dtype)
    
    for branch in tqdm(branches):
        chunk_size = chunks_f[chunks_f['branch'] == branch]['chunk_size_factual'][0]
        chunk = np.zeros(chunk_size, dtype=dtype)
        
        # ('vv_jet1_b_tag', 'f'),
        #    ('vv_jet2_b_tag', 'f'),
        #    ('vv_jet3_b_tag', 'f'),
        #    ('vv_jet4_b_tag', 'f'),
        
        proc_pol = chunks_f[chunks_f['branch'] == branch]['proc_pol'][0]
        pid = processes[processes['proc_pol'] == proc_pol]['pid'][0]
        
        loc = chunks_f[chunks_f['branch'] == 1]['location'][0]
        sid = samples[samples['location'] == loc]['sid'][0]
        
        with ur.open(f'{DATA_ROOT}/{branch}_FinalStates.root:eventTree') as rf:
            chunk['event'] = rf['event'].array()
            chunk['event_category'] = rf['event_category'].array()
        
        with ur.open(f'{DATA_ROOT}/{branch}_PreSelection_qqHH.root:eventTree') as rf:
            #pass_nIsoLeptons = np.array(rf['eventTree']['nIsoLeptons'].array()) == 2
            
            chunk['xx_thrust'] = rf['thrust'].array()
            chunk['xx_e_vis'] = rf['Evis'].array()
            chunk['xx_pt_miss'] = rf['missingPT'].array()
            chunk['xx_nisoleps'] = rf['nIsoLeptons'].array()
            
            chunk['qq_pass'] = rf['preselPassed'].array()

            if kinematics:
                qq_mh1 = rf['lepTypes'].array()
                
                # (excactly) two electron/muon-like signatures:
                pass_ltype11 = np.sum(np.abs(lepTypes) == 11, axis=1) == 2
                pass_ltype13 = np.sum(np.abs(lepTypes) == 13, axis=1) == 2
                
                ll_dilepton_type = pass_ltype11*11 + pass_ltype13*13
                
                # eq: print(np.sum(lepTypes, axis=1))
                
                ll_dilepton_mass_diff = np.array(rf['eventTree']['dileptonMassDiff'].array())
        
        for presel in ['ll', 'vv']:
            with ur.open(f'{DATA_ROOT}/{branch}_PreSelection_{presel}HH.root:eventTree') as rf:
                chunk[f'{presel}_pass'] = rf['preselPassed'].array()
                
                if presel == 'll':                    
                    if kinematics:
                        #pass_nIsoLeptons = np.array(rf['eventTree']['nIsoLeptons'].array()) == 2

                        lepTypes = rf['lepTypes'].array()
                        
                        # (excactly) two electron/muon-like signatures:
                        pass_ltype11 = np.sum(np.abs(lepTypes) == 11, axis=1) == 2
                        pass_ltype13 = np.sum(np.abs(lepTypes) == 13, axis=1) == 2
                        
                        ll_dilepton_type = pass_ltype11*11 + pass_ltype13*13
                        
                        # eq: print(np.sum(lepTypes, axis=1))
                        
                        ll_dilepton_mass_diff = np.array(rf['dileptonMassDiff'].array())
                        # pass_dileptonMassDiff = dileptonMassDiff < 40.
                else:
                    pass
                    
        # TODO: handle kinematics=True
        results = np.append(results, chunk)

    return results
                
        
    #plot_hist(dileptonMass, xlim=[0,500]);

presel_results = presel_stack(DATA_ROOT, samples, processes, chunks_factual, [0, 1])

0
1


/tmp/ipykernel_1098754/228302950.py:61: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  r_size = chunks_f[chunks_f['branch'] == branches]['chunk_size_factual'].sum()


In [ ]:
def get_preselection_per_channel(
    DATA_ROOT:str,
    channel:str,
    version:str='v1')->np.ndarray:
    
    dtype = [
        ('process', '<U60'),
        ('pol_e', 'i'),
        ('pol_p', 'i'),
        ('n_gen', 'i'),
        ('cross_sec', 'f'),
        ('cross_sec_err', 'f'),
        ('n_pass_llhh', 'i'),
        ('n_pass_vvhh', 'i'),
        ('n_pass_qqhh', 'i'),
        ('weight', 'f'),
        ('proc_pol', '<U64')]

    results = np.empty(0, dtype=dtype)
    finished = glob(f'{DATA_ROOT}/*_Source.txt')
    finished.sort()

    for f in (pbar := tqdm(finished)):
        branch = f.split(f'{version}/')[1].split('_Source.txt')[0]
        
        if osp.isfile(f'{DATA_ROOT}/{branch}_FinalStateMeta.json'):
            # Source file is written at the very end -> .root files exist
            
            with open(f, 'r') as file:
                src_spec = file.read()
                if src_spec.startswith('('):
                    src_file, chunk_start, chunk_end = make_tuple(src_spec)
                else:
                    src_file = src_spec
                
            # Read metadata
            with open(f'{DATA_ROOT}/{branch}_FinalStateMeta.json', 'r') as file:
                meta = json.load(file)
                
                # {'crossSection': 133070.796875, 'crossSectionError': 78.4000015258789, 'eventWeight': 1.0, 'nEvtSum': 995, 'polElectron': -1.0, 'polPositron': 1.0, 'processId': 250127, 'processName': '2f_z_bhabhang', 'run': 250127}
                
            n_gen = meta['nEvtSum']
            proc = meta["processName"]
            
            loc, pol = parse_sample_path(src_file)
            procpol = f'{proc}_{get_pol_key(*pol)}'
            
            pbar.set_description(f'Adding {n_gen} events from branch {branch} ({procpol})')
            
            if not procpol in results['proc_pol']:
                results = np.append(results, [np.array([
                    (proc, pol[0], pol[1], n_gen, meta['crossSection'], meta['crossSectionError'], 0, 0, 0, 0., procpol)
                ], dtype=dtype)])
            else:
                results['n_gen'][results['proc_pol'] == procpol] += n_gen
                
            for presel in ['llHH', 'vvHH', 'qqHH']:
                with ur.open(f'{DATA_ROOT}/{branch}_PreSelection_{presel}.root') as rf:
                    passed = rf['eventTree']['preselPassed'].array()
                    
                    n_items = len(passed)
                    if n_items != n_gen:
                        raise Exception('Constraint mismatch')
                    
                    n_passed = np.sum(passed)
                    results[f'n_pass_{presel.lower()}'][results['proc_pol'] == procpol] += n_passed
                    
    for entry in results:
        pol = (entry['pol_e'], entry['pol_p'])
        results['weight'][results['proc_pol'] == entry['proc_pol']] = sample_weight(entry['cross_sec'], pol, entry['n_gen'])

    results = results[np.argsort(results['proc_pol'])]
    
    return results

In [30]:
chunks[chunks['process'] == 'e1e1hh']

array([],
      dtype=[('branch', '<i4'), ('process', '<U60'), ('proc_pol', '<U64'), ('location', '<U512'), ('n_chunks', '<i4'), ('chunk_start', '<i4'), ('chunk_size', '<i4')])

In [33]:
list(filter(lambda x: 'hh' in x, np.unique(chunks['process'])))

['n1n1hh', 'n23n23hh', 'qqhh']